# Loading Tweets into Postgres


- [Testing the process with psycopg2](#section1)
- [Loading all tweets into Postgres](#section2)

Assembling the JSON files with the tweets:

In [1]:
import os

<a id='filenames'></a>

In [91]:
for root, dirs, files in os.walk('/Users/brynstark/Stark gU/DSCI6007-student/FinalProject_BrynStark/tweets/'):
    filenames = files
print(filenames)

['12th st oakland.json', 'bay area.json', 'oakland.json', 'san francisco.json']


<a id='section1'></a>

## Testing the process with psycopg2:

Mimicking [Psycopg2 Documentation](http://initd.org/psycopg/docs/usage.html).

In [11]:
import psycopg2 as pg

In [ ]:
# Connecting to the Postgres database
conn = pg.connect("dbname=bart_twitter user=brynstark")

# Cursor allows operations within Postgres db
cur = conn.cursor()

# To execute Postgres SQL commands:
cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")

# Make changes to db persistent
conn.commit()

# Close connection to cursor and Postgres db
cur.close()
conn.close()

Okay, Postgres with Psycopg2 works.

Let's take the 

In [2]:
with open('tweets/bay area.json') as f:
    tweets_lst = f.read().split('\n')
    print(type(tweets_lst))

<class 'list'>


In [3]:
print(tweets_lst[0])

{"created_at":"Mon Jul 11 02:59:11 +0000 2016","id":752336404441550848,"id_str":"752336404441550848","text":"bay area by my lonely. miss my boo \ud83d\ude1e","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2284778445,"id_str":"2284778445","name":"trmami \u2728","screen_name":"jasiglesias","location":"MIA ","url":null,"description":"let me share this whole new world with you \u2728","protected":false,"verified":false,"followers_count":677,"friends_count":660,"listed_count":2,"favourites_count":8040,"statuses_count":11763,"created_at":"Thu Jan 16 00:12:55 +0000 2014","utc_offset":-25200,"time_zone":"Arizona","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_ba

In [4]:
import json

In [5]:
tweet0 = json.loads(tweets_lst[0])
tweet0

{'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Jul 11 02:59:11 +0000 2016',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 752336404441550848,
 'id_str': '752336404441550848',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': 'bay area by my lonely. miss my boo 😞',
 'timestamp_ms': '1468205951210',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Thu Jan 16 00:12:55 +0000 2014',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'let me share this whole new world with you ✨',
 

In [6]:
tweet0['created_at']

'Mon Jul 11 02:59:11 +0000 2016'

In [7]:
tweet0['text']

'bay area by my lonely. miss my boo 😞'

In [8]:
tweet0['id']

752336404441550848

In [92]:
# Connecting to the Postgres database
conn = pg.connect("dbname=bart_twitter user=brynstark")

# Cursor allows operations within Postgres db
cur = conn.cursor()

In [93]:
# Creating a table
cur.execute('''
            CREATE TABLE tweets_data (
                data json )''')

In [80]:
# Inserting a row of JSON to test
cur.execute('''
            INSERT INTO tweets_data (data)
            VALUES (%s)
            ''', (tweets_lst[0],))

In [81]:
# More testing
cur.execute('''
            INSERT INTO tweets_data (data)
            VALUES (%s)
            ''', (tweets_lst[1],))

In [82]:
# More testing
cur.execute('''
            INSERT INTO tweets_data (data)
            VALUES (%s)
            ''', (tweets_lst[2],))

In [86]:
cur.execute("SELECT * FROM tweets_data")

In [95]:
conn.commit()

In [89]:
# Looking at the results of the query executed above
for record in cur:
    print(record)

({'id': 752336404441550848, 'favorited': False, 'in_reply_to_user_id_str': None, 'retweeted': False, 'geo': None, 'filter_level': 'low', 'contributors': None, 'in_reply_to_user_id': None, 'is_quote_status': False, 'user': {'id': 2284778445, 'url': None, 'following': None, 'description': 'let me share this whole new world with you ✨', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_color': 'C0DEED', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/750809139140452352/kYzNikwN_normal.jpg', 'geo_enabled': True, 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/2284778445/1467841200', 'utc_offset': -25200, 'time_zone': 'Arizona', 'profile_use_background_image': True, 'profile_link_color': '0084B4', 'favourites_count': 8040, 'verified': False, 'friends_count': 660, 'follow_request_sent': None, 'default_profile_image': False, 'is_translator': False, 'profile_sidebar_border_color': '000000', 'followers_co

In [109]:
# Closing connections
cur.close()
conn.close()

Awesome. Now let's get all the tweets in there.

After dropping the 'tweets_data' table I just created...

<a id='section2'></a>

# Loading all tweets into Postgres

List of filenames with tweets defined [above](#filenames).

In [110]:
# Connecting to Postgres
conn = pg.connect("dbname=bart_twitter user=brynstark")
cur = conn.cursor() # Allows us to execute Postgres operations

# Creating a table for all the twitter data
cur.execute('''
            CREATE TABLE tweets_data (
                data json )''')

# For each of the JSON files with Twitter data
for name in filenames:
    
    tweets_lst = []
    path = 'tweets/' + name  # filename.json of tweets
    with open(path) as f:
        tweets_lst = f.read().split('\n')  # Splitting up JSON objects, separated by new-line
    
    counter = 0   # Keep track of tweets added
    lost_tweets = 0   # Keep track of tweets lost
    for tweet in tweets_lst:
        try:
            cur.execute('''
                        INSERT INTO tweets_data (data)
                        VALUES (%s)
                        ''', (tweet,))
            conn.commit()  #  Committing after each execution makes a difference
            counter += 1
        except:
            lost_tweets += 1  # Keep track of tweets that couldn't be added
            continue
    print('From file:', name)
    print('Tweets added', counter)  # Prints the number of tweets added to DB from current JSON file
    print('Tweets lost', lost_tweets)  # Prints tweets not added to DB
    print()
    conn.commit()  # Better safe than sorry

# Closing out connections
conn.commit()   # ABC: Always Be Committing
cur.close()
conn.close()

From file: 12th st oakland.json
Tweets added 2000
Tweets lost 1

From file: bay area.json
Tweets added 479
Tweets lost 1

From file: oakland.json
Tweets added 2662
Tweets lost 1

From file: san francisco.json
Tweets added 2000
Tweets lost 1



Success!! The single tweet that's "lost" for each file is simply the last blank line at the bottom of the file. Perfect!